In [1]:
##https://colab.research.google.com/github/trekhleb/machine-learning-experiments/blob/master/experiments/sketch_recognition_cnn/sketch_recognition_cnn.ipynb#scrollTo=ZTO-30-IMfK3
##how to feed grayscale images to pretrained models which requires 3 input channels:
##https://stackoverflow.com/questions/51995977/how-can-i-use-a-pre-trained-neural-network-with-grayscale-images

from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import math
import datetime
import platform
import pathlib
import random
import pandas as pd
# mogrify

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0:2], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 2 Logical GPU


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# from google.colab import files
# uploaded = files.upload()

In [5]:
from sklearn.model_selection import train_test_split

sketch = pd.read_csv('data/sketch_drawing.csv')

var = 'Unique_expert'
# var = 'Drawing_expert'
# var = 'Useful_expert'
# var = 'Elegence_expert'
# var = 'Creativity_expert'

mapping1 = {'Creativity_expert': 'Creativity',
                       'Useful_expert': 'Useful',
                       'Unique_expert': 'Unique',
                       'Drawing_expert': 'Drawing',
                       'Elegence_expert': 'Elegence',}
mapping = {'Creativity_expert': 'Creativity_expert_round',
                       'Useful_expert': 'Useful_expert_round',
                       'Unique_expert': 'Unique_expert_round',
                       'Drawing_expert': 'Drawing_expert_round',
                       'Elegence_expert': 'Elegence_expert_round',}

In [6]:
sketch1 = pd.read_csv('data/sketch_drawing.csv')
train_df = sketch1[sketch1['set2_'+var] == 'train'] 
val_df = sketch1[sketch1['set2_'+var] == 'val'] 
test_df = sketch1[sketch1['set2_'+var] == 'test'] 

# train_graph = [images[i] for i in train_df.index]
# val_graph = [images[i] for i in val_df.index]
# test_graph = [images[i] for i in test_df.index]

##to add
sketch1 = sketch1.astype({"text": str})
text = list(sketch1['text'])
train_text = [text[i] for i in train_df.index]
val_text = [text[i] for i in val_df.index]
test_text = [text[i] for i in test_df.index]

In [7]:
train_cost = train_df[var]
val_cost = val_df[var]
test_cost = test_df[var]

In [8]:
import os
import shutil
import logging
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal.windows import triang

def get_bin_idx(x):
    return min(int(x * np.float32(10)), 99)

def get_lds_kernel_window(kernel, ks, sigma):
    assert kernel in ['gaussian', 'triang', 'laplace']
    half_ks = (ks - 1) // 2
    if kernel == 'gaussian':
        base_kernel = [0.] * half_ks + [1.] + [0.] * half_ks
        kernel_window = gaussian_filter1d(base_kernel, sigma=sigma) / max(gaussian_filter1d(base_kernel, sigma=sigma))
    elif kernel == 'triang':
        kernel_window = triang(ks)
    else:
        laplace = lambda x: np.exp(-abs(x) / sigma) / (2. * sigma)
        kernel_window = list(map(laplace, np.arange(-half_ks, half_ks + 1))) / max(map(laplace, np.arange(-half_ks, half_ks + 1)))

    return kernel_window

In [9]:
# https://github.com/YyzHarry/imbalanced-regression''

from collections import Counter
from scipy.ndimage import convolve1d

for i in [0]:
    # preds, labels: [Ns,], "Ns" is the number of total samples
#     preds, labels = train_df, train_cost1[var]
    labels = train_cost
    # assign each label to its corresponding bin (start from 0)
    # with your defined get_bin_idx(), return bin_index_per_label: [Ns,] 
    bin_index_per_label = [get_bin_idx(label) for label in labels]

    # calculate empirical (original) label distribution: [Nb,]
    # "Nb" is the number of bins
    Nb = max(bin_index_per_label) + 1
    num_samples_of_bins = dict(Counter(bin_index_per_label))
    emp_label_dist = [num_samples_of_bins.get(i, 0) for i in range(Nb)]

    # lds_kernel_window: [ks,], here for example, we use gaussian, ks=5, sigma=2
    lds_kernel_window = get_lds_kernel_window(kernel='gaussian', ks=5, sigma=2)
    # calculate effective label distribution: [Nb,]
    eff_label_dist = convolve1d(np.array(emp_label_dist), weights=lds_kernel_window, mode='constant')

    # Use re-weighting based on effective label distribution, sample-wise weights: [Ns,]
    eff_num_per_label = [eff_label_dist[bin_idx] for bin_idx in bin_index_per_label]
    weights = [np.float32(1 / x) for x in eff_num_per_label]
    # # calculate loss
    # loss = weighted_mse_loss(preds, labels, weights=weights)                  

In [10]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [11]:
# pip install tensorflow_text

In [16]:
import tensorflow_hub as hub
import tensorflow_text as text
bert_preprocess_model = hub.KerasLayer("model weight/bert_en_uncased_preprocess_3")

In [41]:
# text_preprocessed_train = bert_preprocess_model(train_text)
# text_preprocessed_val = bert_preprocess_model(val_text)
# text_preprocessed_test = bert_preprocess_model(test_text)

# text_test = ['this is such an amazing movie!']
# text_preprocessed = bert_preprocess_model(text_test)

# print(f'Keys       : {list(text_preprocessed.keys())}')
# print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
# print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
# print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
# print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

In [14]:
# pip install tf-models-official

In [51]:
# from tensorflow.keras.optimizers import RMSprop, Adam, SGD
# from tensorflow.keras.callbacks import LearningRateScheduler
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras import Model, Input, layers, regularizers
# from keras.models import load_model
# from tensorflow.keras import activations
# from official.nlp import optimization 

# epochs = 1000
# steps_per_epoch = 56
# num_train_steps = steps_per_epoch * epochs

# ##unique: 0, drawing: 0.05, elegance: 0.05, creativity: 0.05, useful: 0.1
# a = 0.005
# # if var == 'Unique_expert':
# #     a = 0
# # elif var == 'Creativity_expert':
# #     a = 0.02
# # elif var == 'Useful_expert':
# #     a = 0.1
# num_warmup_steps = int(a*num_train_steps)
# init_lr = 2e-5
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

# text_input = [Input(shape=(), dtype=tf.string, name='text')]
# preprocessor = hub.load("model weight/bert_en_uncased_preprocess_3")
# tokenize = hub.KerasLayer(preprocessor.tokenize)
# tokenized_inputs = [tokenize(segment) for segment in text_input]
# seq_length = 32
# bert_pack_inputs = hub.KerasLayer(
#     preprocessor.bert_pack_inputs,
#     arguments=dict(seq_length=seq_length))  # Optional argument.
# encoder_inputs = bert_pack_inputs(tokenized_inputs)
# encoder = hub.KerasLayer("model weight/small_bert_bert_en_uncased_L-4_H-512_A-8_2", trainable=True, name='BERT_encoder')
# outputs = encoder(encoder_inputs)
# net = outputs['pooled_output']
# # net = layers.Dropout(0.1)(net)
# net = layers.Dense(64, activation='relu')(net)
# net = layers.Dropout(0.1)(net)
# out1y = layers.Dense(1, activation='relu', name='out1y')(net)
# model = Model(text_input, out1y)

# model.compile(optimizer=optimizer,
#               loss=tf.keras.losses.MeanSquaredError(),
#               metrics= tf.keras.metrics.RootMeanSquaredError())

In [52]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model, Input, layers, regularizers
from keras.models import load_model
from tensorflow.keras import activations
from official.nlp import optimization 

epochs = 1000
steps_per_epoch = 56
num_train_steps = steps_per_epoch * epochs

##unique: 0, drawing: 0.05, elegance: 0.05, creativity: 0.05, useful: 0.1
a = 0.05
# if var == 'Unique_expert':
#     a = 0
# elif var == 'Creativity_expert':
#     a = 0.02
# elif var == 'Useful_expert':
#     a = 0.1
num_warmup_steps = int(a*num_train_steps)
init_lr = 2e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

text_input = Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer("model weight/bert_en_uncased_preprocess_3", name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer("model weight/small_bert_bert_en_uncased_L-4_H-512_A-8_2", trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
# net = layers.Dropout(0.1)(net)
net = layers.Dense(64, activation='relu')(net)
net = layers.Dropout(0.1)(net)
out1y = layers.Dense(1, activation='relu', name='out1y')(net)
model = Model(text_input, out1y)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics= tf.keras.metrics.RootMeanSquaredError())

In [53]:
# model.summary()

In [54]:
##definition of callbacks
epochs = 1000
steps_per_epoch = 56
batch_size = 24

early_stopping_callback = tf.keras.callbacks.EarlyStopping(\
    patience=20,
    monitor="val_loss",
    restore_best_weights=True,
    verbose=1)

In [55]:
training_history = model.fit(
    x = [tf.constant(train_text)],
    y = {'out1y':np.array(train_cost)},
    sample_weight={'out1y': np.array(weights)},
    epochs=epochs,
    batch_size = batch_size,
    steps_per_epoch=steps_per_epoch,
    validation_data=(
        [tf.constant(val_text)],
        {'out1y':np.array(val_cost)}),
    callbacks=[early_stopping_callback]
)

Epoch 1/1000
56/56 [==============================] - 18s 257ms/step - loss: 0.0763 - root_mean_squared_error: 2.1277 - val_loss: 1.9718 - val_root_mean_squared_error: 1.4042
Epoch 2/1000
56/56 [==============================] - 14s 243ms/step - loss: 0.0339 - root_mean_squared_error: 1.4557 - val_loss: 1.8431 - val_root_mean_squared_error: 1.3576
Epoch 3/1000
56/56 [==============================] - 14s 242ms/step - loss: 0.0319 - root_mean_squared_error: 1.3892 - val_loss: 1.8913 - val_root_mean_squared_error: 1.3752
Epoch 4/1000
56/56 [==============================] - 13s 241ms/step - loss: 0.0254 - root_mean_squared_error: 1.2630 - val_loss: 1.6480 - val_root_mean_squared_error: 1.2837
Epoch 5/1000
56/56 [==============================] - 14s 256ms/step - loss: 0.0218 - root_mean_squared_error: 1.1977 - val_loss: 1.5332 - val_root_mean_squared_error: 1.2382
Epoch 6/1000
56/56 [==============================] - 14s 242ms/step - loss: 0.0187 - root_mean_squared_error: 1.1227 - val_l

56/56 [==============================] - 12s 206ms/step - loss: 0.0027 - root_mean_squared_error: 0.4370 - val_loss: 1.3411 - val_root_mean_squared_error: 1.1581
Epoch 48/1000
56/56 [==============================] - 12s 210ms/step - loss: 0.0025 - root_mean_squared_error: 0.4248 - val_loss: 1.3633 - val_root_mean_squared_error: 1.1676
Epoch 49/1000
56/56 [==============================] - 13s 233ms/step - loss: 0.0025 - root_mean_squared_error: 0.4281 - val_loss: 1.3842 - val_root_mean_squared_error: 1.1765
Epoch 50/1000
56/56 [==============================] - 13s 237ms/step - loss: 0.0024 - root_mean_squared_error: 0.4224 - val_loss: 1.3579 - val_root_mean_squared_error: 1.1653
Epoch 51/1000
56/56 [==============================] - 13s 234ms/step - loss: 0.0022 - root_mean_squared_error: 0.4056 - val_loss: 1.3587 - val_root_mean_squared_error: 1.1656
Epoch 52/1000
56/56 [==============================] - 13s 238ms/step - loss: 0.0024 - root_mean_squared_error: 0.4084 - val_loss: 1.3

In [56]:
idx = 0
n = 2
# a = 0.5
output = pd.DataFrame()

test_results0 = model.predict(tf.constant(test_text))
test_result= test_results0.reshape(len(test_text)).tolist()
test_cost = test_df[var]

difference_array = np.subtract(test_result, test_cost)
squared_array = np.square(difference_array)
mse = squared_array.mean()

correlation_matrix = np.corrcoef(test_cost, test_result)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

output.loc[idx,'n'] = n
output.loc[idx,'var'] = var
output.loc[idx,'lr'] = init_lr
output.loc[idx,'mse'] = mse
output.loc[idx,'r^2'] = r_squared

In [57]:
output

,n,var,lr,mse,r^2
0,2.0,Unique_expert,0.00002,1.289976,0.423234


In [102]:
##save and load models
##https://stackoverflow.com/questions/46610732/how-to-freeze-some-layers-when-fine-tune-resnet50

initial_lrate = 0.000005

# var = 'Unique_expert'
# var = 'Drawing_expert'
# var = 'Useful_expert'
# var = 'Elegence_expert'
# var = 'Creativity_expert'

# model_name = 'model weight/'+var+'/new sketch-'+var+'-best.h5'
# model_name = 'model weight/'+var+'/both_ratio-'+str(a)+'-'+var+'-best.h5'
# model_name = 'model weight/'+var+'/simple_joint_ratio-'+str(a)+'-'+var+'-best.h5'
# model_name = 'model weight/'+var+'/joint_lr-'+str(initial_lrate)+'-'+var+'-best.h5'
model_name = 'model weight/'+var+'/new text-'+var+'-best.h5'
# model.save(model_name, save_format='h5')

from keras.models import load_model
model = load_model(model_name, custom_objects={'KerasLayer':hub.KerasLayer,'AdamWeightDecay': optimizer})

In [103]:
# model.layers

In [104]:
sketch1 = sketch1.astype({"text": str})
text = list(sketch1['text'])

from keras import backend as K
sec_last = model.layers[2]
get_output = K.function([model.layers[0].input, ],[sec_last.output, model.layers[-1].output])
[emb_text, predictions] = get_output(tf.constant(text))

np.savetxt('text emb 1-'+var, emb_text['pooled_output'])

In [22]:
# ##to test which BEET
# idx = 0
# output = pd.DataFrame()
# for key in map_name_to_handle.keys():
#     bert_model_name = key
#     tfhub_handle_encoder = map_name_to_handle[bert_model_name]
#     tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]   
#     text_input = Input(shape=(), dtype=tf.string, name='text')
#     preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
#     encoder_inputs = preprocessing_layer(text_input)
#     encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
#     outputs = encoder(encoder_inputs)
#     net = outputs['pooled_output']
#     # net = layers.Dropout(0.1)(net)
#     net = layers.Dense(128, activation='relu')(net)
#     net = layers.Dropout(0.1)(net)
#     out1y = layers.Dense(1, activation='relu', name='out1y')(net)
#     model = Model(text_input, out1y)

#     model.compile(optimizer=optimizer,
#                   loss=tf.keras.losses.MeanSquaredError(),
#                   metrics= tf.keras.metrics.RootMeanSquaredError())
#     training_history = model.fit(
#         x = tf.constant(train_text),
#         y = {'out1y':np.array(train_cost)},
#         sample_weight={'out1y': np.array(weights)},
#         epochs=epochs,
#         batch_size = batch_size,
#         steps_per_epoch=steps_per_epoch,
#         validation_data=(
#             tf.constant(val_text),
#             {'out1y':np.array(val_cost)}),
#         callbacks=[early_stopping_callback]
#     )

#     test_results0 = model.predict(tf.constant(test_text))
#     test_result= test_results0.reshape(len(test_text)).tolist()
#     test_cost = test_df[var]

#     difference_array = np.subtract(test_result, test_cost)
#     squared_array = np.square(difference_array)
#     mse = squared_array.mean()

#     correlation_matrix = np.corrcoef(test_cost, test_result)
#     correlation_xy = correlation_matrix[0,1]
#     r_squared = correlation_xy**2

#     output.loc[idx,'n'] = idx
#     output.loc[idx,'var'] = var
#     output.loc[idx,'lr'] = init_lr
#     output.loc[idx,'mse'] = mse
#     output.loc[idx,'r^2'] = r_squared
#     output.loc[idx,'model'] = key
#     idx += 1
#     print (key+":  ", r_squared)